# AG2: RAG

Retrieval-Augmented Generation **(RAG)** is a technique to improve LLM-generated responses by providing additional knowledge. This typically involves gathering the information and injecting it into an agent’s system message for use by their LLM.<br>
There are a number of ways to incorporate RAG into your AG2 workflow and agents:
- Use an AG2 reference agent, **DocAgent**
- Add RAG capabilities to an agent
- Incorporate context into an agent’s system message (manually and automatically)

DocAgent
- https://docs.ag2.ai/docs/user-guide/reference-agents/docagent

## SETUP

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables (for API key)
load_dotenv()

# Set up OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("Please set the OPENAI_API_KEY environment variable or add it to a .env file")

# Define the model to use
MODEL_GPT = "gpt-4o-mini"

## 1. DocAgent

Use AG2’s reference agent, **DocAgent**, built specifically for RAG. It will take the hassle out of loading, parsing, storing, and querying documents/web pages.
- https://docs.ag2.ai/docs/user-guide/reference-agents/docagent

## 2. Add RAG capabilities to an agent

### RAG Capability: Neo4j GraphRAG

Based on AG2’s base GraphRAG capability, this Neo4j GraphRAG capability allows the embedding and querying of information with a Neo4j graph database.<br>
See the **Using Neo4j’s graph database with AG2 agents for Q&A notebook**.
- https://docs.ag2.ai/docs/use-cases/notebooks/notebooks/agentchat_graph_rag_neo4j

### RAG Capability: FalkorDB GraphRAG

Also based on AG2’s base GraphRAG capability, this capability uses a FalkorDB GraphRAG database.<br>
See the **Trip planning with a FalkorDB GraphRAG agent using a Swarm notebook**.
- https://docs.ag2.ai/docs/use-cases/notebooks/notebooks/agentchat_swarm_graphrag_trip_planner

## 3. Incorporating context into an Agent’s system message

In [2]:
import os
from typing import Any

from autogen import ConversableAgent

config_list = {"api_type": "openai", "model": "gpt-4o", "api_key": os.environ["OPENAI_API_KEY"]}

base_system_message = "You are a helpful agent, answering questions about the files in a directory:\n{filelisting}"

In [3]:
# Attach to the hook, it gets all files in the current directory and updates the associated agent’s system message accordingly

def give_agent_file_listing(agent: ConversableAgent, messages: list[dict[str, Any]]) -> None:
    # Get the list of files in the current directory
    files = os.listdir()

    # Put them in a string
    files_str = "\n".join(files)

    # Use the system message template and update the agent's system message to include the file listing
    agent.update_system_message(base_system_message.format(filelisting=files_str))

In [4]:
# Create the agent and attach the hook

files_agent = ConversableAgent(
    name="files_agent",
    system_message="""You are a helpful agent, answering questions about the files in a directory.""",
    llm_config=config_list,
    )

files_agent.register_hook(
    hookable_method="update_agent_state",
    hook=give_agent_file_listing,
    )

In [5]:
# Create a human-in-the-loop agent and ask our files_agent about the files

human = ConversableAgent(
    name="human",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=1,
    )

In [7]:
chat_result = human.initiate_chat(
    recipient=files_agent,
    message="Tell me about the files in my directory.",
    max_turns=1,
    )

human (to files_agent):

Tell me about the files in my directory.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
files_agent (to human):

Your directory contains a variety of files and folders, here's a general breakdown:

1. **Hidden directories:**
   - `.cache`, `.env`, `.git`, `.ipynb_checkpoints`, `.venv`: These are directories that typically store environment settings, version control information, or caching data. 
   - `.gitignore`: A configuration file for Git to specify which files or directories should be ignored during version control.

2. **Jupyter Notebook files:**
   - `101-ag2-quick-start.ipynb` through `113-ag2-advanced-1.ipynb`: These files are Jupyter Notebook files, usually with Python or other programming code, likely related to some sort of learning or tutorial sequence about "ag2" content.

3. **Plain directories:**
   - `001`, `002`: These could be general directories, used to organize related files 

In [8]:
print(chat_result.summary)

Your directory contains a variety of files and folders, here's a general breakdown:

1. **Hidden directories:**
   - `.cache`, `.env`, `.git`, `.ipynb_checkpoints`, `.venv`: These are directories that typically store environment settings, version control information, or caching data. 
   - `.gitignore`: A configuration file for Git to specify which files or directories should be ignored during version control.

2. **Jupyter Notebook files:**
   - `101-ag2-quick-start.ipynb` through `113-ag2-advanced-1.ipynb`: These files are Jupyter Notebook files, usually with Python or other programming code, likely related to some sort of learning or tutorial sequence about "ag2" content.

3. **Plain directories:**
   - `001`, `002`: These could be general directories, used to organize related files or information.

4. **Documentation:**
   - `README.md`: This is often a markdown file used to document information about your project like how to set it up or run it.

5. **Requirements files:**
   - `r

In [9]:
print(len(chat_result.chat_history))
# print(chat_result.chat_history[0])
# print(chat_result.chat_history[-1])
print(chat_result.chat_history)

2
[{'content': 'Tell me about the files in my directory.', 'role': 'assistant', 'name': 'human'}, {'content': 'Your directory contains a variety of files and folders, here\'s a general breakdown:\n\n1. **Hidden directories:**\n   - `.cache`, `.env`, `.git`, `.ipynb_checkpoints`, `.venv`: These are directories that typically store environment settings, version control information, or caching data. \n   - `.gitignore`: A configuration file for Git to specify which files or directories should be ignored during version control.\n\n2. **Jupyter Notebook files:**\n   - `101-ag2-quick-start.ipynb` through `113-ag2-advanced-1.ipynb`: These files are Jupyter Notebook files, usually with Python or other programming code, likely related to some sort of learning or tutorial sequence about "ag2" content.\n\n3. **Plain directories:**\n   - `001`, `002`: These could be general directories, used to organize related files or information.\n\n4. **Documentation:**\n   - `README.md`: This is often a markd